In [ ]:
x=['sports/ipl2020/news','sports/football','sports/tennis','sports/badminton','sports/athletics','sports/other-sports','india','fact-check','lifestyle/health','technology/news','technology/talking-points','data-intelligence-unit','cities','education-today','education-today/gk-and-current-affairs','education-today/notification','education-today/jobs-and-careers','trending-news','binge-watch','world']

In [ ]:
len(x)

20

In [ ]:
from bs4 import BeautifulSoup
import urllib.request,sys,time
import requests
import pandas as pd
filename = "/content/drive/MyDrive/data/news.csv"
f = open(filename, "a", encoding='utf-8')
headers = "Id,Title,Summary,Category,\n"
f.write(headers)
news_ = ['sports/ipl2020/news','sports/football','sports/tennis','sports/badminton','sports/athletics','sports/other-sports','india','fact-check','lifestyle/health','technology/news','technology/talking-points','data-intelligence-unit','cities','education-today','education-today/gk-and-current-affairs','education-today/notification','education-today/jobs-and-careers','trending-news','binge-watch','world']
i = -1
sr_no = 1
upperframe = []
for num in range(15*len(news_)):
  if(num%15 == 0):
    i=i+1
  url = 'https://www.indiatoday.in/' + str(news_[i]) +'?page='+str(num%15)
  print(url)
  page = requests.get(str(url))
  time.sleep(1)
  soup = BeautifulSoup(page.text, 'html.parser')
  frame = []
  links = soup.find_all('div', attrs={'class':'detail'})
  for j in links:
    news_id = str(sr_no)
    title = j.find("h2").find('a').text.strip()
    try:
      summary = j.find("p").text.strip()
    except:
      summary = ''
    category = str(news_[i])
    frame.append((news_id,title,summary,category))
    title=title.replace("\n", "")
    summary=summary.replace("\n", "")
    f.write(news_id.replace(",", "^") + "," + title.replace(",", "^") + "," + summary.replace(",", "^") + "," + category.replace(",", "^") + "," + url+"\n")
    sr_no += 1
  upperframe.extend(frame)
f.close()


In [21]:
import pandas as pd
data = pd.read_csv("/content/drive/MyDrive/data/news.csv")
data.head()

,Id,Title,Summary,Category,Unnamed: 4
0,1,Kohli is absolutely not what you see on the cr...,Adam Zampa opened up about his first day at Ro...,sports/ipl2020/news,https://www.indiatoday.in/sports/ipl2020/news?...
1,2,David Warner^ Virat Kohli miss out on Deep Das...,Pragyan Ojha and Deep Dasgupta had the task of...,sports/ipl2020/news,https://www.indiatoday.in/sports/ipl2020/news?...
2,3,Have no issues with Kohli: Suryakumar on 'star...,Suryakumar Yadav and Virat Kohli's stare incid...,sports/ipl2020/news,https://www.indiatoday.in/sports/ipl2020/news?...
3,4,Left training mid-way^ didn't feel like eating...,Suryakumar Yadav missed out on the plane to Au...,sports/ipl2020/news,https://www.indiatoday.in/sports/ipl2020/news?...
4,5,Sehwag is outspoken of his dislike for me: Max...,Kings XI Punjab batsman Glenn Maxwell has resp...,sports/ipl2020/news,https://www.indiatoday.in/sports/ipl2020/news?...


In [3]:
data.shape

(6841, 5)

In [4]:
import re
def transform(sen):
    sentence = re.sub('[^a-zA-Z]', ' ', sen)
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)
    sentence = re.sub(r'\s+', ' ', sentence)
    return sentence

In [5]:
for i in range(len(data['Summary'])):
  sen=str(data['Summary'][i])
  data['Summary'][i] = transform(sen)
for i in range(len(data['Title'])):
  sen=str(data['Title'][i])
  data['Title'][i] = transform(sen)

In [6]:
for i in range(len(data['Category'])):
  data['Category'][i]=data['Category'][i].replace("/"," ")

In [7]:
data.head()

,Id,Title,Summary,Category,Unnamed: 4
0,1,Kohli is absolutely not what you see on the cr...,Adam Zampa opened up about his first day at Ro...,sports ipl2020 news,https://www.indiatoday.in/sports/ipl2020/news?...
1,2,David Warner Virat Kohli miss out on Deep Dasg...,Pragyan Ojha and Deep Dasgupta had the task of...,sports ipl2020 news,https://www.indiatoday.in/sports/ipl2020/news?...
2,3,Have no issues with Kohli Suryakumar on stare ...,Suryakumar Yadav and Virat Kohli stare inciden...,sports ipl2020 news,https://www.indiatoday.in/sports/ipl2020/news?...
3,4,Left training mid way didn feel like eating af...,Suryakumar Yadav missed out on the plane to Au...,sports ipl2020 news,https://www.indiatoday.in/sports/ipl2020/news?...
4,5,Sehwag is outspoken of his dislike for me Maxw...,Kings XI Punjab batsman Glenn Maxwell has resp...,sports ipl2020 news,https://www.indiatoday.in/sports/ipl2020/news?...


In [8]:
news = data['Summary']

In [9]:
news[1]

'Pragyan Ojha and Deep Dasgupta had the task of selecting playing XIs each one as their IPL XI and another that they felt had the chance of beating time champions Mumbai Indians '

In [10]:
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [11]:
def words(news):
  from nltk.corpus import stopwords
  from nltk.tokenize import word_tokenize
  from nltk.stem import WordNetLemmatizer
  for i in range(len(news)):
    news[i]=word_tokenize(news[i])
    for j in range(len(news[i])):
      if news[i][j].lower() in my_stopwords:
        news[i][j] =""
  unwanted = {''}
  for i in range(len(news)):
    news[i] = [e for e in news[i] if e not in unwanted]
  lemmatizer = WordNetLemmatizer()
  for i in range(len(news)):
    for j in range(len(news[i])):
      news[i][j] = lemmatizer.lemmatize(news[i][j])
  x=vector_creation(news);

In [22]:
def vector_creation(news):
  from gensim.models import Word2Vec,KeyedVectors
  gn_path = r'/content/drive/MyDrive/data/GoogleNews-vectors-negative300.bin'
  model = KeyedVectors.load_word2vec_format(gn_path, binary=True)
  from gensim.models.fasttext import FastText
  embedding_size = 300
  window_size = 5
  min_word = 5
  down_sampling = 1e-2
  fast_Text_model  = FastText(news,size=embedding_size,window=window_size,min_count=min_word,sample=down_sampling,workers = 4,sg=1,iter=100)
  dummy = 300*[0]
  vec = 300*[0]
  news_vec = []
  count =0
  for i in range(len(news)):
    for j in news[i]:
      try:
        vec= vec+model[j]
      except:
        try:
          count = count+1
          vec= vec+fast_Text_model[j]
        except:
          vec= vec+dummy
          continue
        continue
    news_vec.append(vec)
  return (news_vec);


In [ ]:
filename = "/content/drive/MyDrive/data/news_vec.csv"
f = open(filename, "a", encoding='utf-8')
news_vec=vector_creation(news);
for i in range(len(news_vec)):
  for j in range(300):
    f.write(str(news_vec[i][j]).replace("\n",'') + ",")
  f.write("\n")
f.close

In [15]:
import pandas as pd
x = pd.read_csv("/content/drive/MyDrive/data/news_vec.csv")

In [16]:
x.columns = range(len(x.columns))
x['Title'] = data['Title']
x['Category'] = data['Category']
x['Link'] = data['Unnamed: 4']
x = x.columns.to_frame().T.append(x, ignore_index=True)
x.to_csv('/content/drive/MyDrive/data/Final_news_vectors.csv')

In [44]:
def reco(vec):
  import numpy as np
  import pandas as pd
  data_vec = pd.read_csv('/content/drive/MyDrive/data/Final_news_vectors.csv')
  data = pd.read_csv("/content/drive/MyDrive/data/news.csv")
  data_vec=data_vec.drop(['Unnamed: 0','300'],axis=1)
  data_vec=data_vec.drop([0],axis=0)
  diff = data_vec[data_vec.columns[~data_vec.columns.isin(['Title','Category','Link'])]]
  dist = []
  count=0
  for i in range(300):
    if (diff[str(i)].dtype != 'float64'):
      diff[str(i)][0]=diff[str(i)][0][:-2]
      diff[str(i)].astype(float)
  link = []
  title = []
  summary = []
  for i in range(diff.shape[0]):
    if np.linalg.norm(diff.loc[1000] - diff.loc[i+1]) < 100:
      link.append(data['Title'][i])
      title.append(data['Unnamed: 4'][i])
      summary.append(data['Summary'][i])
  return link,title,summary

In [49]:
link,title,summary=reco(x.iloc[2]);
for i in range(len(link)):
  print(link[i],"\n",title[i],"\n",summary[i])
  print("------------------------------------------------------------------")

Indian wrestlers aiming for four medals at Tokyo Olympics: WFI President 
 https://www.indiatoday.in/sports/other-sports?page=7 
 The WFI president Brij Bhushan Sharan believes that the Indian wrestlingcontingent can bring home at least four medals from the Tokyo Olympics.
------------------------------------------------------------------
UFC 257: Poirier avenges 2014 defeat with 2nd-round knockout of McGregor 
 https://www.indiatoday.in/sports/other-sports?page=7 
 Conor McGregor got off to a strong start in the first round before beingbrought down by Dustin Poirier mid-way into the second round in the mainevent of UFC 257 on Sunday.
------------------------------------------------------------------
IOC planning on getting all athletes vaccinated ahead of Tokyo games 
 https://www.indiatoday.in/sports/other-sports?page=7 
 The International Olympic Committee (IOC) is working with the World HealthOrganization (WHO) to get all athletes^ involved in the upcoming Olympics2021^ vaccinated 

In [ ]:
import numpy as np
from flask import Flask, request, render_template
#import spacy
from flask_ngrok import run_with_ngrok
doc_idx = 0
user_idx = 0
user_new = np.zeros((1,20))
app = Flask(_name_, template_folder= 'NLP_Recommender')
run_with_ngrok(app)


@app.route('/', methods=['GET', 'POST'])
def homepage():
  return render_template('index2.html')

@app.route('/newUser_page', methods=['GET', 'POST'])
def newUser_page():
  output_new_user, user_topic_matrix1 = newUserHybridRecommender(new_user_vec, user_topic_matrix, doc_topic_matrix)
  user_new = user_topic_matrix1[-1]
  user_idx = user_topic_matrix1.shape[0]
  return render_template('index3.html',user_id = user_idx, data1 = output_new_user[0][1],data2 = output_new_user[1][1],data3 = output_new_user[2][1],data4 = output_new_user[3][1],data5 = output_new_user[4][1],data6 = output_new_user[5][1],data7 = output_new_user[6][1],data8 = output_new_user[7][1],data9 = output_new_user[8][1],data10 = output_new_user[9][1])


@app.route('/existingUser', methods=['GET', 'POST'])
def existingUser_page():
  return render_template('index4.html') #, data=request.form['text'], response=doc)

@app.route('/existingUser_headLiness', methods=['GET', 'POST'])
def existingUser_headLiness():
  user_idx =int(request.form.get('text'))
  print(user_topic_matrix[user_idx-1])
  output_new_user1,kk = newUserHybridRecommender(user_topic_matrix[user_idx-1], user_topic_matrix, doc_topic_matrix)
  print(output_new_user1)
  return render_template('index3.html', user_id = user_idx, data1 = output_new_user1[0][1],data2 = output_new_user1[1][1],data3 = output_new_user1[2][1],data4 = output_new_user1[3][1],data5 = output_new_user1[4][1],data6 = output_new_user1[5][1],data7 = output_new_user1[6][1],data8 = output_new_user1[7][1],data9 = output_new_user1[8][1],data10 = output_new_user1[9][1])



@app.route('/activeUser_headLiness', methods=['GET', 'POST'])
def existingUser_headLiness():
  output_new_user2 = oldUserHybridRecommender(user_idx, doc_id, user_topic_matrix, doc_topic_matrix)
  print(user_topic_matrix[user_idx-1])
  print(output_new_user2)
  return render_template('index3.html', user_id = user_idx, data1 = output_new_user2[0][1],data2 = output_new_user2[1][1],data3 = output_new_user2[2][1],data4 = output_new_user2[3][1],data5 = output_new_user2[4][1],data6 = output_new_user2[5][1],data7 = output_new_user2[6][1],data8 = output_new_user2[7][1],data9 = output_new_user2[8][1],data10 = output_new_user2[9][1])

@app.route('/content_page',methods=['GET','POST'])
def content_page():
  if request.method == "POST":
    if request.form.get("1"):
      final_data = articlesnew[0][2]
      doc_id = articlesnew[0][0]
    elif request.form.get("2"):
      final_data = articlesnew[1][2]
      doc_id = articlesnew[1][0]
    elif request.form.get("3"):
      final_data = articlesnew[2][2]
      doc_id = articlesnew[2][0]
    elif request.form.get("4"):
      final_data = articlesnew[3][2]
      doc_id = articlesnew[3][0]
    elif request.form.get("5"):
      final_data = articlesnew[4][2]
      doc_id = articlesnew[4][0]
    elif request.form.get("6"):
      final_data = articlesnew[5][2]
      doc_id = articlesnew[5][0]
    elif request.form.get("7"):
      final_data = articlesnew[6][2]
      doc_id = articlesnew[6][0]
    elif request.form.get("8"):
      final_data = articlesnew[7][2]
      doc_id = articlesnew[7][0]
    elif request.form.get("9"):
      final_data = articlesnew[8][2]
      doc_id = articlesnew[8][0]
    elif request.form.get("10"):
      final_data = articlesnew[9][2]
      doc_id = articlesnew[9][0]
    return render_template('index5.html',summary=final_data)

if _name_ == '_main_':
  app.run()